# Properties y herencia

## Sobrescritura de getters

Ahora vamos a modificar la clase ContenedorDeMercancias para añadir los atributos altura y anchura, ya que estos serán los mismos para todos los tipos de contenedores. Los utilizaremos para sacar el **volumen** del contenedor.

También vamos a añadir longitud como atributo, por lo que es necesario modificar el inicializador y los métodos factory:

In [1]:
import iso6346

In [2]:
class ContenedorDeMercancias:
    
    # Definición de constantes de clase
    
    ALTURA = 8.5
    ANCHURA = 8.0
    
    # Definición de atributos de clase
    
    siguiente_codigo_de_serie = 1337
    
    @staticmethod
    def _crear_codigo_BIC(codigo_propietario, codigo_de_serie):
        return iso6346.create(
        owner_code=codigo_propietario,
        serial=str(codigo_de_serie).zfill(6))
    
    @classmethod
    def _generar_codigo_de_serie(cls):
        
        resultado = cls.siguiente_codigo_de_serie 
        cls.siguiente_codigo_de_serie += 1
        return resultado
    
    # Crea un contenedor vacío
    @classmethod
    def crear_contenedor_vacio(cls, codigo_propietario, longitud, **kwargs):
        return cls(codigo_propietario, longitud, contenidos = [], **kwargs)
    
    # Crea un contenedor a partir de un codigo de propietario y unos items
    @classmethod
    def crear_contenedor_con_elementos(cls, codigo_propietario, longitud, contenidos, **kwargs):
        return cls(codigo_propietario, longitud, list(contenidos), **kwargs)
        
    
    def __init__(self, codigo_propietario, longitud, contenidos, **kwargs):
        
        # Definición de atributos de instancia
        
        self.codigo_propietario = codigo_propietario
        self.longitud = longitud
        self.contenidos = contenidos
        self.bic = ContenedorDeMercancias._crear_codigo_BIC(
            codigo_propietario=codigo_propietario,
            codigo_de_serie=ContenedorDeMercancias._generar_codigo_de_serie()
        )
        
    
    @property
    def volumen(self):
        return ContenedorDeMercancias.ALTURA * ContenedorDeMercancias.ANCHURA * self.longitud

In [3]:
c = ContenedorDeMercancias.crear_contenedor_vacio("YML", longitud=20)

In [4]:
c.volumen

1360.0

Para que funcione también en la clase ContenedorDeMercanciasRefrigerado, debemos adaptar su inicializador para que acepte el argumento **longitud**:

In [5]:
class ContenedorDeMercanciasRefrigerado(ContenedorDeMercancias):
    
    MAX_CELSIUS = 4.0
        
    def __init__(self, codigo_propietario, longitud, contenidos, *, celsius, **kwargs):
        super().__init__(codigo_propietario, longitud, contenidos, **kwargs)
        
        if celsius > ContenedorDeMercanciasRefrigerado.MAX_CELSIUS:
            raise ValueError("La temperatura es demasiado alta")
        
        # Renombramos _celsius a celsius, autoencapsulándolo y accediendo a través de su property
        
        self.celsius = celsius
        
    # Creamos estos dos métodos para pasar de celsius a farenheit y viceversa
    
    @staticmethod
    def _celsius_to_farenheit(celsius):
        return celsius * 9/5 + 32
    
    @staticmethod
    def _farenheit_to_celsius(farenheit):
        return (farenheit -32) * 5/9 
    
    @property
    def celsius(self):
        return self._celsius
    
    @celsius.setter
    def celsius(self, valor):
        
        if valor > ContenedorDeMercanciasRefrigerado.MAX_CELSIUS:
            raise ValueError("La temperatura es demasiado alta")
            
        self._celsius = valor
        
    # Creamos los getter y setter de farenheit
    
    @property
    def farenheit(self):
        return ContenedorDeMercanciasRefrigerado._celsius_to_farenheit(self.celsius)
    
    @farenheit.setter
    def farenheit(self, value):
        self.celsius = ContenedorDeMercanciasRefrigerado._farenheit_to_celsius(value)
        
    
    
    @staticmethod
    def _crear_codigo_bic(codigo_propietario, codigo_de_serie):
        return iso6346.create(
        owner_code=codigo_propietario,
        serial=str(codigo_de_serie).zfill(6),
        category="R")

In [6]:
r = ContenedorDeMercanciasRefrigerado.crear_contenedor_vacio("YML", longitud=20, celsius=3)

In [7]:
r.volumen

1360.0

Sin embargo, hay que tener en cuenta que la máquina refrigeradora del contenedor ocupa 100 metros cubicos de espacio, por lo que hay que restarselo al volumen total del contenedor.

Para ello vamos a crear una constante de clase llamada **REFRIGERATOR_VOLUME** y sobrescribir la property **volumen**, adaptándola al contenedor:

In [8]:
class ContenedorDeMercanciasRefrigerado(ContenedorDeMercancias):
    
    MAX_CELSIUS = 4.0
    VOLUMEN_MAQUINA_REFRIGERADORA = 100
        
    def __init__(self, codigo_propietario, longitud, contenidos, *, celsius, **kwargs):
        super().__init__(codigo_propietario, longitud, contenidos, **kwargs)
        
        if celsius > ContenedorDeMercanciasRefrigerado.MAX_CELSIUS:
            raise ValueError("La temperatura es demasiado alta")
        
        # Renombramos _celsius a celsius, autoencapsulándolo y accediendo a través de su property
        
        self.celsius = celsius
        
    # Creamos estos dos métodos para pasar de celsius a farenheit y viceversa
    
    @staticmethod
    def _celsius_to_farenheit(celsius):
        return celsius * 9/5 + 32
    
    @staticmethod
    def _farenheit_to_celsius(farenheit):
        return (farenheit -32) * 5/9 
    
    @property
    def celsius(self):
        return self._celsius
    
    @celsius.setter
    def celsius(self, valor):
        
        if valor > ContenedorDeMercanciasRefrigerado.MAX_CELSIUS:
            raise ValueError("La temperatura es demasiado alta")
            
        self._celsius = valor
        
    # Creamos los getter y setter de farenheit
    
    @property
    def farenheit(self):
        return ContenedorDeMercanciasRefrigerado._celsius_to_farenheit(self.celsius)
    
    @farenheit.setter
    def farenheit(self, value):
        self.celsius = ContenedorDeMercanciasRefrigerado._farenheit_to_celsius(value)
    
    @staticmethod
    def _crear_codigo_bic(codigo_propietario, codigo_de_serie):
        return iso6346.create(
        owner_code=codigo_propietario,
        serial=str(codigo_de_serie).zfill(6),
        category="R")
    
    @property
    def volumen(self):
        return super().volumen - ContenedorDeMercanciasRefrigerado.VOLUMEN_MAQUINA_REFRIGERADORA

In [9]:
r = ContenedorDeMercanciasRefrigerado.crear_contenedor_vacio("YML", longitud=20, celsius=3)

In [10]:
r.volumen

1260.0

## Sobrescritura de setters

Para realizar la sobrescritura de setters, será necesario crear una nueva clase: **ContenedorDeMercanciasRefrigeradoConCalefaccion**. Este tipo de contenedores permiten tanto enfriar como calentar cuando sea necesario.

Vamos a asumir que dichos contenedores no deben caer nunca por debajo de temperaturas de -20 grados celsius.

Para sobrescribir la property setter de la clase base, debemos hacer referencia a ella en el decorador de la property:

In [11]:
class ContenedorDeMercanciasRefrigeradoConCalefaccion(ContenedorDeMercanciasRefrigerado):
    
    MIN_CELSIUS = -20.0
    
    @ContenedorDeMercanciasRefrigerado.celsius.setter
    def celsius(self, valor):
        
        if valor < ContenedorDeMercanciasRefrigeradoConCalefaccion.MIN_CELSIUS:
            raise ValueError("La temperatura es demasiado baja")
        
        """
        Esto es necesario para llamar al property setter de la clase base:
        
        @celsius.setter
        def celsius(self, valor):

            if valor > ContenedorDeMercanciasRefrigerado.MAX_CELSIUS:
                raise ValueError("La temperatura es demasiado alta")

            self._celsius = valor
        """
        
        ContenedorDeMercanciasRefrigerado.celsius.fset(self, valor)
            

### Importante

Además, como podemos ver dentro del setter, para invocar al setter de la clase base es necesario utilizar el método fset del objeto devuelto por la property.

Esto se debe a que dicho objeto mantiene referencias a los métodos getter y setter de la clase llamados fget y fset, respectivamente.

In [12]:
h = ContenedorDeMercanciasRefrigeradoConCalefaccion.crear_contenedor_vacio("YML", longitud=20, celsius=3)

In [13]:
try:
    h.celsius = -21
except ValueError as error:
    print(error)

La temperatura es demasiado baja


In [14]:
try:
    h.celsius = 5
except ValueError as error:
    print(error)

La temperatura es demasiado alta
